In [1]:
pip install ultralytics supervision opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.4/217.4 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
import supervision as sv
import warnings
warnings.filterwarnings("ignore")

# ==============================
# PATHS
# ==============================
VIDEO_DIR = "/kaggle/input/datasets/anushreeu04/chain-snatching-video-set1"
OUTPUT_CSV = "/kaggle/working/all_videos_tracking.csv"

# ==============================
# LOAD MODELS
# ==============================
model = YOLO("yolov8n.pt")
tracker = sv.ByteTrack()

all_data = []

# ==============================
# LOOP THROUGH ALL VIDEOS
# ==============================
for video_file in os.listdir(VIDEO_DIR):
    if not video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
        continue
    
    video_path = os.path.join(VIDEO_DIR, video_file)
    print("Processing:", video_file)
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        continue
    
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    frame_id = 0
    temp_data = []
    
    # ---------- TRACKING ----------
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_id += 1
        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        
        # Filter for person class (class_id == 0)
        detections = detections[detections.class_id == 0]
        detections = tracker.update_with_detections(detections)
        
        for bbox, tid, conf in zip(detections.xyxy, detections.tracker_id, detections.confidence):
            if tid is None:
                continue
            
            x1, y1, x2, y2 = map(int, bbox)
            temp_data.append([video_file, frame_id, int(tid), x1, y1, x2, y2, float(conf)])
    
    cap.release()
    
    # ---------- DATAFRAME ----------
    df = pd.DataFrame(
        temp_data,
        columns=["video_name", "frame", "track_id", "x1", "y1", "x2", "y2", "confidence"]
    )
    
    if df.empty:
        continue
    
    # ---------- FEATURE ENGINEERING ----------
    # Center coordinates
    df["cx"] = (df["x1"] + df["x2"]) / 2
    df["cy"] = (df["y1"] + df["y2"]) / 2
    
    # Bounding box dimensions
    df["width"] = df["x2"] - df["x1"]
    df["height"] = df["y2"] - df["y1"]
    df["area"] = df["width"] * df["height"]
    
    # Initialize anomaly features
    df["speed"] = 0.0
    df["acceleration"] = 0.0
    df["direction_change"] = 0.0
    df["area_change"] = 0.0
    
    # ---------- PER-TRACK FEATURES ----------
    for tid in df["track_id"].unique():
        track_df = df[df["track_id"] == tid].sort_values("frame").copy()
        
        if len(track_df) < 3:  # Need at least 3 frames for meaningful features
            continue
        
        # Calculate displacement and speed
        dx = track_df["cx"].diff()
        dy = track_df["cy"].diff()
        displacement = np.sqrt(dx**2 + dy**2)
        
        # Speed
        speed_values = displacement.fillna(0).values
        df.loc[track_df.index, "speed"] = speed_values
        
        # Acceleration (change in speed)
        accel_values = np.abs(np.diff(speed_values, prepend=speed_values[0]))
        df.loc[track_df.index, "acceleration"] = accel_values
        
        # Direction changes (sudden changes in movement angle)
        angles = np.arctan2(dy.values, dx.values)
        angle_diff = np.abs(np.diff(angles))
        # Normalize angle differences to [0, π]
        angle_diff = np.minimum(angle_diff, 2*np.pi - angle_diff)
        # Prepend 0 for the first frame
        direction_values = np.concatenate([[0], angle_diff])
        df.loc[track_df.index, "direction_change"] = direction_values
        
        # Area changes (size variations)
        area_values = track_df["area"].values
        area_change_values = np.abs(np.diff(area_values, prepend=area_values[0])) / (area_values + 1e-6)
        df.loc[track_df.index, "area_change"] = area_change_values
    
    # ---------- ANOMALY DETECTION ----------
    # Calculate baseline statistics (using median and MAD for robustness)
    speed_median = df["speed"].median()
    speed_mad = np.median(np.abs(df["speed"] - speed_median))
    if speed_mad == 0:
        speed_mad = 1.0  # Avoid division by zero
    
    accel_median = df["acceleration"].median()
    accel_mad = np.median(np.abs(df["acceleration"] - accel_median))
    if accel_mad == 0:
        accel_mad = 1.0
    
    direction_median = df["direction_change"].median()
    direction_mad = np.median(np.abs(df["direction_change"] - direction_median))
    if direction_mad == 0:
        direction_mad = 1.0
    
    # Define thresholds (adjust multipliers based on your data)
    speed_threshold = speed_median + 3 * speed_mad
    accel_threshold = accel_median + 3 * accel_mad
    direction_threshold = direction_median + 2.5 * direction_mad
    
    # Multi-criteria anomaly detection
    df["speed_anomaly"] = (df["speed"] > speed_threshold).astype(int)
    df["accel_anomaly"] = (df["acceleration"] > accel_threshold).astype(int)
    df["direction_anomaly"] = (df["direction_change"] > direction_threshold).astype(int)
    
    # Combined anomaly score (at least 2 out of 3 criteria)
    df["anomaly_score"] = (
        df["speed_anomaly"] + 
        df["accel_anomaly"] + 
        df["direction_anomaly"]
    )
    
    # Detect chain snatching: high anomaly score OR extreme speed
    extreme_speed_threshold = speed_median + 5 * speed_mad
    df["chain_snatching_detected"] = (
        (df["anomaly_score"] >= 2) | 
        (df["speed"] > extreme_speed_threshold)
    ).astype(int)
    
    # Smooth detection using rolling window to reduce false positives
    for tid in df["track_id"].unique():
        track_mask = df["track_id"] == tid
        track_indices = df[track_mask].index
        smoothed = df.loc[track_indices, "chain_snatching_detected"].rolling(
            window=5, min_periods=1
        ).mean()
        df.loc[track_indices, "chain_snatching_detected"] = (smoothed >= 0.4).astype(int)
    
    all_data.append(df)
    
    # Print per-video summary
    detection_rate = df["chain_snatching_detected"].mean() * 100
    print(f"  ✓ {video_file}: {detection_rate:.1f}% anomalous frames detected")

# ==============================
# SAVE FINAL CSV
# ==============================
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_csv(OUTPUT_CSV, index=False)
    print("\n✅ Combined CSV generated:", OUTPUT_CSV)
    print("Total videos processed:", final_df["video_name"].nunique())
    print(f"Total frames tracked: {len(final_df)}")
    print(f"Total unique tracks: {final_df['track_id'].nunique()}")
    
    # Summary statistics
    print("\n📊 Overall Detection Summary:")
    overall_detection_rate = final_df["chain_snatching_detected"].mean() * 100
    print(f"  Overall anomaly rate: {overall_detection_rate:.1f}%")
    
    print("\n📹 Per-Video Summary:")
    for video in final_df["video_name"].unique():
        video_df = final_df[final_df["video_name"] == video]
        detection_rate = video_df["chain_snatching_detected"].mean() * 100
        num_tracks = video_df["track_id"].nunique()
        print(f"  {video}: {detection_rate:.1f}% anomalous | {num_tracks} people tracked")
else:
    print("❌ No data processed")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Processing: video7.mp4
  ✓ video7.mp4: 12.9% anomalous frames detected
Processing: video4.mp4
  ✓ video4.mp4: 9.2% anomalous frames detected
Processing: video1.mp4
  ✓ video1.mp4: 18.4% anomalous frames detected
Processing: video6.mp4
  ✓ video6.mp4: 21.4% anomalous frames detected
Processing: video5.mp4
  ✓ video5.mp4: 14.0% anomalous frames detected
Processing: video3.mp4
  ✓ video3.mp4: 21.1% anomalous frames detected
Processing: video8.mp4
  ✓ video8.mp4: 18.2% anomalous frames detected
Processing: video2.mp4
  ✓ video2.mp4: 39.5% anomalous frames detected

✅ Combined CSV generated: /kaggle/working/all_videos_tracking.csv
Total videos processed: 8
Total frames tracked: 12480
T